In diesem notebook wurde gemessen, in welchen Dialogen Uneinigkeit herrscht. Die Posts aus diesen Dialogen wurden mit 1 gelabelt, die ohne Uneinigkeit mit 0.


In [2]:
import pandas as pd

In [3]:
data_full = pd.read_excel('data_full_clean.xlsx')
data_full


,Unnamed: 0.2,Unnamed: 0,1:,2: Unnamed: 0.1,3: Unnamed: 0,4: tree_id,5: post_id,6: parent_id,7: author_id,8: in_reply_to_user_id,9: text,10: created_at,11: path,12: a/b_author,13: author_flow,label_tone,label_opinion,var_tone,var_opinion,Unnamed: 0.1
0,0,0,0,0,41136,928468196608405632,928468196608405632,NaN,5734902,NaN,Jamaika-Sondierungen: Özdemir fordert mehr Ent...,2017-11-09 03:44:02.000000 +00:00,1,root,a,NaN,NaN,-99,-99,NaN
1,1,1,1,1,41143,928468196608405632,928498731271577600,9.284682e+17,721321723719761920,5.734902e+06,@tagesschau Ganz ehrlich jetzt mal was allgeme...,2017-11-09 05:45:22.000000 +00:00,1,a,b,Ton ggü. Anderen\postiv\Höflich/respektvoll,Meinung\Kontra-Migration\Nationalzentriert,1,-1,NaN
2,2,2,2,2,41144,928468196608405632,928555684903424000,9.284987e+17,87278692,7.213217e+17,@beatrix_doring @tagesschau Sie haben wohl kei...,2017-11-09 09:31:41.000000 +00:00,1,b,c,Ton ggü. Anderen\neagtiv\sarkastisch,"Meinung\on Topic, nicht erkennbar",-3,0,NaN
3,3,3,3,3,41145,928468196608405632,928620432596533120,9.285557e+17,721321723719761920,8.727869e+07,@Piratenbaer @tagesschau Natürlich habe ich Fa...,2017-11-09 13:48:58.000000 +00:00,1,a,b,NaN,NaN,-99,-99,NaN
4,4,4,880,880,566659,928468196608405632,928468196608405632,NaN,5734902,NaN,Jamaika-Sondierungen: Özdemir fordert mehr Ent...,2017-11-09 03:44:02.000000 +00:00,2,root,a,NaN,NaN,-99,-99,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971,16,375,290,290,79134,1543618374755835904,1543622965732037120,1.543623e+18,1518678425216684032,1.542228e+18,"@KlaudioKlau1 @eliyojkb jargon, schon mal gehört?",2022-07-03 15:49:28.000000 +00:00,3,b,b,Ton ggü. Anderen\neagtiv\sarkastisch,Meinung\nicht on Topic,-3,-10,375.0
972,17,376,291,291,79137,1543618374755835904,1543623390644379904,1.543623e+18,1542227574889251072,1.518678e+18,@0711Piedro @eliyojkb Nicht jede Verkommenheit...,2022-07-03 15:51:10.000000 +00:00,3,a,c,Ton ggü. Anderen\neagtiv\in Rage,Meinung\nicht on Topic,-2,-10,376.0
973,18,377,292,292,79144,1543618374755835904,1543623701391884032,1.543623e+18,1518678425216684032,1.542228e+18,@KlaudioKlau1 @eliyojkb bist du die sprachpoli...,2022-07-03 15:52:24.000000 +00:00,3,b,b,Ton ggü. Anderen\neagtiv\sarkastisch,Meinung\nicht on Topic,-3,-10,377.0
974,19,378,293,293,79145,1543618374755835904,1543623882623667968,1.543624e+18,1542227574889251072,1.518678e+18,@0711Piedro @eliyojkb Mehr als du Möchtegern-L...,2022-07-03 15:53:07.000000 +00:00,3,a,c,Ton ggü. Anderen\neagtiv\beleidigend,Meinung\nicht on Topic,-5,-10,378.0


In [4]:
def disagreement(data):
    data['disagreement'] = None
    data_on_topic = data[data['var_opinion']>-9]
    data_on_topic = data_on_topic[data_on_topic['var_opinion']!=0]
    grouped_df = data_on_topic.groupby(['4: tree_id', '11: path'])
    for index, group in grouped_df:
        authors = group.groupby(['13: author_flow'])
        path = group['11: path'].tolist()[0]
        id = group['4: tree_id'].tolist()[0]
        if authors.ngroups > 2:
            group = group[group['6: parent_id'].notna()]
            authors = group.groupby(['13: author_flow'])
        op_values = {}
        for i, author in authors:
            op_value = sum(author['var_opinion'].tolist())/len(author)
            op_values[i[0]] = op_value
        if len(op_values)==2:
            val_a = list(op_values.values())[0]
            val_b = list(op_values.values())[1]
            if (val_a<0 and val_b<0) or (val_a>0 and val_b>0):
                disagree = 0
            if (val_a<0 and val_b>0) or (val_a>0 and val_b<0):
                disagree = 1 
        else:
            continue
        data.loc[(data['4: tree_id'] == id) & (data['11: path'] == path), 'disagreement'] = disagree
    return data
    
        


dis_df = disagreement(data_full)

In [5]:
dis_df.head(20)

,Unnamed: 0.2,Unnamed: 0,1:,2: Unnamed: 0.1,3: Unnamed: 0,4: tree_id,5: post_id,6: parent_id,7: author_id,8: in_reply_to_user_id,...,10: created_at,11: path,12: a/b_author,13: author_flow,label_tone,label_opinion,var_tone,var_opinion,Unnamed: 0.1,disagreement
0,0,0,0,0,41136,928468196608405632,928468196608405632,NaN,5734902,NaN,...,2017-11-09 03:44:02.000000 +00:00,1,root,a,NaN,NaN,-99,-99,NaN,None
1,1,1,1,1,41143,928468196608405632,928498731271577600,9.284682e+17,721321723719761920,5.734902e+06,...,2017-11-09 05:45:22.000000 +00:00,1,a,b,Ton ggü. Anderen\postiv\Höflich/respektvoll,Meinung\Kontra-Migration\Nationalzentriert,1,-1,NaN,None
2,2,2,2,2,41144,928468196608405632,928555684903424000,9.284987e+17,87278692,7.213217e+17,...,2017-11-09 09:31:41.000000 +00:00,1,b,c,Ton ggü. Anderen\neagtiv\sarkastisch,"Meinung\on Topic, nicht erkennbar",-3,0,NaN,None
3,3,3,3,3,41145,928468196608405632,928620432596533120,9.285557e+17,721321723719761920,8.727869e+07,...,2017-11-09 13:48:58.000000 +00:00,1,a,b,NaN,NaN,-99,-99,NaN,None
4,4,4,880,880,566659,928468196608405632,928468196608405632,NaN,5734902,NaN,...,2017-11-09 03:44:02.000000 +00:00,2,root,a,NaN,NaN,-99,-99,NaN,1
5,5,5,881,881,566666,928468196608405632,928498731271577600,9.284682e+17,721321723719761920,5.734902e+06,...,2017-11-09 05:45:22.000000 +00:00,2,a,b,Ton ggü. Anderen\postiv\Höflich/respektvoll,Meinung\Kontra-Migration\Nationalzentriert,1,-1,NaN,1
6,6,6,882,882,566667,928468196608405632,928555684903424000,9.284987e+17,87278692,7.213217e+17,...,2017-11-09 09:31:41.000000 +00:00,2,b,c,Ton ggü. Anderen\neagtiv\sarkastisch,"Meinung\on Topic, nicht erkennbar",-3,0,NaN,1
7,7,7,883,883,566669,928468196608405632,928620644450947200,9.285557e+17,721321723719761920,8.727869e+07,...,2017-11-09 13:49:49.000000 +00:00,2,a,b,Ton ggü. Anderen\neagtiv\sarkastisch,Meinung\Kontra-Migration\Nationalzentriert,-3,-1,NaN,1
8,8,8,884,884,566670,928468196608405632,928816338159718400,9.286206e+17,87278692,7.213217e+17,...,2017-11-10 02:47:26.000000 +00:00,2,b,c,Ton ggü. Anderen\neagtiv\in Rage,Meinung\Pro-Migration\Humanitär,-2,4,NaN,1
9,9,9,885,885,566671,928468196608405632,928851944038391808,9.288163e+17,721321723719761920,8.727869e+07,...,2017-11-10 05:08:55.000000 +00:00,2,a,b,Ton ggü. Anderen\neagtiv\resigniert,Meinung\nicht on Topic,-1,-10,NaN,1


In [6]:
df = data_full[data_full['4: tree_id']==1652564083688848896]
df = df[df['11: path']==7]
df


,Unnamed: 0.2,Unnamed: 0,1:,2: Unnamed: 0.1,3: Unnamed: 0,4: tree_id,5: post_id,6: parent_id,7: author_id,8: in_reply_to_user_id,...,10: created_at,11: path,12: a/b_author,13: author_flow,label_tone,label_opinion,var_tone,var_opinion,Unnamed: 0.1,disagreement
932,953,953,857,857,60180,1652564083688848896,1652564083688848896,NaN,4164240143,NaN,...,2023-04-30 06:42:35.000000 +00:00,7,root and a,a,Ton ggü. Anderen\neagtiv\sarkastisch,Meinung\Kontra-Migration\Rassistisch/Vorurteil...,-3,-3,NaN,1
933,954,954,858,858,60196,1652564083688848896,1652913874805432064,1.652564e+18,1339355066147467008,4.164240e+09,...,2023-05-01 05:52:32.000000 +00:00,7,a,b,Ton ggü. Anderen\neagtiv\in Rage,Meinung\Pro-Migration\Rassismuskritisch,-2,3,NaN,1
934,955,955,859,859,60197,1652564083688848896,1652924210241469952,1.652914e+18,405354250,1.339355e+18,...,2023-05-01 06:33:36.000000 +00:00,7,b,c,Ton ggü. Anderen\postiv\Höflich/respektvoll,"Meinung\on Topic, nicht erkennbar",1,0,NaN,1
935,956,956,860,860,60202,1652564083688848896,1652938323151692032,1.652924e+18,1339355066147467008,4.053542e+08,...,2023-05-01 07:29:41.000000 +00:00,7,a,b,Ton ggü. Anderen\neagtiv\in Rage,Meinung\Pro-Migration\Rassismuskritisch,-2,3,NaN,1
936,957,957,861,861,60203,1652564083688848896,1652939683528072960,1.652938e+18,405354250,1.339355e+18,...,2023-05-01 07:35:05.000000 +00:00,7,b,c,Ton ggü. Anderen\neagtiv\sarkastisch,Meinung\Kontra-Migration\Anti-Wokeness,-3,-2,NaN,1
937,958,958,862,862,60204,1652564083688848896,1652941165841249024,1.652940e+18,1339355066147467008,4.053542e+08,...,2023-05-01 07:40:58.000000 +00:00,7,a,b,Ton ggü. Anderen\neagtiv\deligitimierend,Meinung\nicht on Topic,-4,-10,NaN,1
938,959,959,863,863,60205,1652564083688848896,1652942405287436032,1.652941e+18,405354250,1.339355e+18,...,2023-05-01 07:45:54.000000 +00:00,7,b,c,Ton ggü. Anderen\neagtiv\sarkastisch,Meinung\nicht on Topic,-3,-10,NaN,1


In [7]:
dis_df.to_excel('data_disagreement.xlsx')